# Data Importation and Pre-processing

In [ ]:
#Import data

from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String
from sqlalchemy import sql, select, join, desc
from sqlalchemy import inspect

# Create a Engine object which is our handle into the database.
engine = create_engine('sqlite:////blue/bsc4452/share/Class_Files/data/world.sqlite')

# Connect to the database
conn = engine.connect() #conextion

In [ ]:
# Read the metadata from the existing database.
DBInfo=MetaData(engine)

In [ ]:
#Check tables present in engine
inspector = inspect(engine)
schemas = inspector.get_schema_names()
print(inspector.get_table_names())

In [ ]:
# Auto-create the city object basedon the metadata read into the DBInfo.
country=Table('country', DBInfo, autoload=True)
city=Table('city', DBInfo, autoload=True)
language=Table('countrylanguage', DBInfo, autoload=True)

# Function: Continent Language Information

This function provides information about the langauge spoken in a country. The user will provide the name of the continent, then select a name of country from the list provided. The continent and country name will be verified and returns the country's:
* all spoken language(s)
* official language(s)
* most spoken language and
* other countries where the most spoken langauge is spoken on the continent.

In [ ]:
print("------------------------------- Continent Info -------------------------------")
continent_lst = ['Africa','Asia','Europe','North America','South America','Oceania']

#Check if country is present in the list
while True:
    #print("-------------------------")
    print("Enter the name of any continent of your choice.")
    name = input("Enter Africa, Asia,Europe,North America, Oceania or South America: ")
    name.lower()
    continent = name.title()
    if continent not in continent_lst:
        print(f"{continent} not found. Try again!")
    else:
        print(f"{continent} found!")
        break

#Query data
query1=(select([country.c.Name,country.c.Continent,language.c.Language,language.c.IsOfficial,language.c.Percentage])\
        .select_from(country.join(language))\
        .where(country.c.Continent == continent))
cLang = conn.execute(query1)

cLang1 = []
for row in cLang:
    cLang1.append(row)

#Function for getting information summary about a continent
def continentSummary(cLang1):
    country_lst = []
    for index in range(len(cLang1)):
        if cLang1[index][0] not in country_lst: 
            country_lst.append(cLang1[index][0])
    country_lst.sort()
    return(country_lst)

print("------------------------------- Continent Summary -------------------------------")
countryLst = continentSummary(cLang1)
print(countryLst)


#Check if country is present in the list
while True:
    print("-------------------------")
    name = input("Select a country name from the country list above: ")
    cName = name.title()
    if cName not in countryLst:
        print(f"{cName} not found. Try again!")
    else:
        print(f"{cName} found!")
        break


cName_lst = []
offLang = []
hLang = 0
otherCountries = []

for index in range(len(cLang1)):
    #print()
    if cName == cLang1[index][0]:
        cName_lst.append(cLang1[index][2])
        offLang_T = cLang1[index][3]
        try:
            if offLang_T == 'T':
                offLang.append(cLang1[index][2])
        except:
            continue
        perLang = cLang1[index][4]
        try:
            if perLang > hLang:
                hLang = perLang
                mostSpokenLang = cLang1[index][2:]
                #print(mostSpokenLang)
        except:
            continue
    if mostSpokenLang[0] == cLang1[index][2] and cName != cLang1[index][0]:
        otherCountries.append(cLang1[index][0])
    else:
        continue

#print(mostSpokenLang)
#print(otherCountries)

print(f"Languages spoken in {cName} are: {cName_lst}.")
print("--------")
if len(offLang) == 0:
    print(f"{cName} has no official language.")
else:
    print(f"The official language in {cName} is {offLang}.")
print("--------")
print(f"The most spoken language in {cName} is {mostSpokenLang[0]} by {round(mostSpokenLang[2],0)}% of the population.")
print("--------")
if len(otherCountries) == 0:
    print(f"{mostSpokenLang[0]} is not spoken in other countries in {continent}.")
else:
    print(f"{mostSpokenLang[0]} is spoken in {otherCountries} in {continent}.")

